In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install ipython numpy tqdm
!pip install transformers==4.30.2

Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.3.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.30.2 whi

In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
os.chdir('/content/drive/MyDrive/CCL/DISC')
print(os.getcwd())

/content/drive/MyDrive/CCL/DISC


In [4]:
from IPython.display import display, HTML
import torch
import numpy as np
from tqdm import tqdm
from src.utils.model_util import load_model_from_checkpoint
from src.model.read_comp_triflow import ReadingComprehensionDetector as DetectorMdl
from config import Config as config
from demo_helper.data_processor import DataHandler
import nltk
import json

# **1. Set and prepare input sentences**

In [5]:
with open('/content/drive/MyDrive/CCL/DISC/train_sent_idiom.json', "r", encoding="utf-8") as f:
    sent_idiom = json.load(f)
with open('/content/drive/MyDrive/CCL/DISC/train_sent_literal.json', "r", encoding="utf-8") as f:
    sent_literal = json.load(f)
with open('/content/drive/MyDrive/CCL/DISC/train_pie_idiom.json', "r", encoding="utf-8") as f:
    pie_idiom = json.load(f)
with open('/content/drive/MyDrive/CCL/DISC/train_pie_literal.json', "r", encoding="utf-8") as f:
    pie_literal = json.load(f)
print(sent_idiom)
print(sent_literal)
print(pie_idiom)
print(pie_literal)

['It took a lot of elbow grease to get the old engine running again.', "It's a constant battle for us, as he is a morning person and I am a night owl, so I find that going to sleep at 9.30 really cuts out the best hours.", 'Even the somewhat seedy failed private eye has a heart of gold (and a bad hairstyle).', "ESA's Space Weather Office is like Europe's stellar agony aunt, offering forecasts, advice and information for any organisation, individual or institution vulnerable to space phenomena.", 'This aged, rich wine is no shrinking violet with its with peppery, meaty focus and its broad yet tannic palatal impressions.', 'They are a boogeyman he scares you with while he colludes with your enemies, turns your country into a banana republic and uses the office to enrich himself.', "If you want dirt, the private eye suggested, you should steal the man's garbage.", "The result is that whether you sleep on your back or your side, the ultimate night's sleep is no longer a pipe dream.", "This

# **2. Load model**

In [6]:
data_handler = DataHandler(config)
detector_model= load_model_from_checkpoint(DetectorMdl, data_handler.config)

Loading Pre-trained Glove Embeddings...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers grea

Loading model from /content/drive/MyDrive/CCL/DISC/checkpoints/ReadComp_magpie_random_cross_attn-glove-char-pos-tri_latest.mdl
=> loading checkpoint '/content/drive/MyDrive/CCL/DISC/checkpoints/ReadComp_magpie_random_cross_attn-glove-char-pos-tri_latest.mdl'


/content/drive/MyDrive/CCL/DISC/src/utils/model_util.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filename)


=> loaded checkpoint '/content/drive/MyDrive/CCL/DISC/checkpoints/ReadComp_magpie_random_cross_attn-glove-char-pos-tri_latest.mdl'


In [7]:
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [8]:
data_idiom = data_handler.prepare_input(sent_idiom, pie_idiom)
data_literal = data_handler.prepare_input(sent_literal, pie_literal)


# **3. Model inference**

In [9]:
with torch.no_grad():
  pie_embedding_idiom = detector_model(data_idiom)
  pie_embedding_literal = detector_model(data_literal)
pie_embedding_idiom = pie_embedding_idiom.cpu().detach().numpy()
pie_embedding_literal = pie_embedding_literal.cpu().detach().numpy()

# **4. Output**

In [10]:
embedding_idiom_dict = dict(zip(pie_idiom, pie_embedding_idiom))
embedding_literal_dict = dict(zip(pie_literal, pie_embedding_literal))
# iterate the dictionaries to check the output
for key, value in embedding_idiom_dict.items():
    print(f"{key}, {value.shape}")
for key, value in embedding_literal_dict.items():
    print(f"{key}, {value.shape}")

elbow grease, (512,)
night owl, (512,)
heart of gold, (512,)
agony aunt, (512,)
shrinking violet, (512,)
banana republic, (512,)
private eye, (512,)
pipe dream, (512,)
rocket science, (512,)
nest egg, (512,)
bull market, (512,)
beached whale, (512,)
lounge lizard, (512,)
bear market, (512,)
white hat, (512,)
smoking gun, (512,)
old flame, (512,)
ivory tower, (512,)
black sheep, (512,)
gravy train, (512,)
rat race, (512,)
spring chicken, (512,)
inner circle, (512,)
bad apple, (512,)
honey trap, (512,)
open book, (512,)
baby blues, (512,)
brain surgery, (512,)
red flag, (512,)
white elephant, (512,)
rat run, (512,)
graveyard shift, (512,)
dirty money, (512,)
high life, (512,)
pain in the neck, (512,)
guinea pig, (512,)
cat's eyes, (512,)
low-hanging fruit, (512,)
busy bee, (512,)
green fingers, (512,)
ancient history, (512,)
devil's advocate, (512,)
piece of cake, (512,)
brass ring, (512,)
apples and oranges, (512,)
ghost town, (512,)
flower child, (512,)
copy cat, (512,)
secret santa, (

In [12]:
# save the embeddings in .pt
torch.save(embedding_idiom_dict, "/content/drive/MyDrive/CCL/DISC/DISC_emb/disc_pie_embeddings_idioms.pt")
torch.save(embedding_literal_dict, "/content/drive/MyDrive/CCL/DISC/DISC_emb/disc_pie_embeddings_literal.pt")